In [162]:
import pickle as pickle
import os
import pandas as pd
import torch

# additional
from collections import defaultdict
import numpy as np
# 
import sys
sys.path.append('/opt/ml/level2-klue-level2-nlp-03/KSY')
from load_data import *
label_list = ['no_relation', 'org:top_members/employees', 'org:members',
                  'org:product', 'per:title', 'org:alternate_names',
                  'per:employee_of', 'org:place_of_headquarters', 'per:product',
                  'org:number_of_employees/members', 'per:children',
                  'per:place_of_residence', 'per:alternate_names',
                  'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
                  'per:spouse', 'org:founded', 'org:political/religious_affiliation',
                  'org:member_of', 'per:parents', 'org:dissolved',
                  'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
                  'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
                  'per:religion']

In [163]:
train_data_dir = "/opt/ml/level2-klue-level2-nlp-03/baseline/dataset/train/train.csv"
# original type data
train_dataset = load_data(train_data_dir)
train_dataset.head()

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,'비틀즈','조지 해리슨',no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,'민주평화당','대안신당',no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,'광주FC','한국프로축구연맹',org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,'아성다이소','박정부',org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,'요미우리 자이언츠','1967',no_relation


### Sentence에 Entity typing v0.1

In [11]:
def en_load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)

In [14]:
en_train_data_dir = "./data/en_all_train.csv"
en_train_pd = pd.read_csv(en_train_data_dir)

In [15]:
en_train_pd.head()

,id,sentence,subject_entity,object_entity,label,source,en_trans_sentence
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia,"""Something""은 조지 해리슨가 작곡하고 비틀즈의 1969년 앨범 Abbey ..."
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree,호남 기반 바른미래당·대안신당·민주평화당은 우여곡절 끝에 민생당(가칭)으로 다시 태...
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree,K리그2 1위를 달리고 있는 광주FC은 26일 한국프로축구연맹로부터 '풀스타디움상'...
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree,흥공가생활용품점아성다이소(대표 박정부)은 코로나19 바이러스로 어려움을 겪고 있는 ...
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia,1967 프로야구 드래프트 1순위로 요미우리 자이언츠와 함께 1967에 합류한 후 ...


In [164]:
train_data_dir = "/opt/ml/level2-klue-level2-nlp-03/baseline/dataset/train/train.csv"
pd_dataset = pd.read_csv(train_data_dir)
# start_idx부터 글자 시작, end_idx는 마지막 글자
subj = eval(pd_dataset.iloc[0].subject_entity)
print('subj',subj)
obj = eval(pd_dataset.iloc[0].object_entity)
print('obj',obj)
sents = pd_dataset.iloc[0].sentence
print(sents)

subj {'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}
obj {'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}
〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.


In [52]:
# subj, obj 중에 뭐가 먼저 나오는지 알 수 없음   
new_subj = '[E1]' + sents[subj['start_idx'] : subj['end_idx']+1] + '[/E1]'
new_obj = '[E2]' + sents[obj['start_idx'] : obj['end_idx']+1] + '[/E2]'
print(new_subj, new_obj)

[E1]비틀즈[/E1] [E2]조지 해리슨[/E2]


In [49]:
if subj['start_idx'] > obj['start_idx']:
    temp = sents[:obj['start_idx']] + new_obj + sents[obj['end_idx']+1 : subj['start_idx']] + new_subj + sents[subj['end_idx']+1 :]
print(temp)

〈Something〉는 [E2]조지 해리슨[/E2]이 쓰고 [E1]비틀즈[/E1]가 1969년 앨범 《Abbey Road》에 담은 노래다.


In [54]:
subj = eval(pd_dataset.iloc[2].subject_entity)
print('subj',subj)
obj = eval(pd_dataset.iloc[2].object_entity)
print('obj',obj)
sents = pd_dataset.iloc[2].sentence
print(sents)

subj {'word': '광주FC', 'start_idx': 21, 'end_idx': 24, 'type': 'ORG'}
obj {'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx': 41, 'type': 'ORG'}
K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.


In [56]:
new_subj = '[E1]' + sents[subj['start_idx'] : subj['end_idx']+1] + '[/E1]'
new_obj = '[E2]' + sents[obj['start_idx'] : obj['end_idx']+1] + '[/E2]'
if subj['start_idx'] < obj['start_idx']:
    temp = sents[:subj['start_idx']] + new_subj + sents[subj['end_idx']+1 : obj['start_idx']] + new_obj + sents[obj['end_idx']+1 :]
print(temp)

K리그2에서 성적 1위를 달리고 있는 [E1]광주FC[/E1]는 지난 26일 [E2]한국프로축구연맹[/E2]으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.


In [165]:
def preprocessing_ent_dataset(dataset):
  # """ 처음 불러온 csv 파일에서 sentecen에 등장하고 start_idx, end_idx 를 갖는 subject, object를 marking."""
    subject_entity = []
    object_entity = []
    subject_type = []
    object_type = []
    # subject_entity : {'word':- , 'start_idx':-, 'end_idx':-}
    for idx, (subj, obj, sents) in enumerate(zip(dataset['subject_entity'] ,dataset['object_entity'], dataset['sentence'])):
        subj, obj = eval(subj), eval(obj)
        new_subj = '[E1]' + sents[subj['start_idx'] : subj['end_idx']+1] + '[/E1]'
        new_obj = '[E2]' + sents[obj['start_idx'] : obj['end_idx']+1] + '[/E2]'
        if subj['start_idx'] >= obj['start_idx']:
            temp = sents[:obj['start_idx']] + new_obj + sents[obj['end_idx']+1 : subj['start_idx']] + new_subj + sents[subj['end_idx']+1 :]
        elif subj['start_idx'] < obj['start_idx']:
            temp = sents[:subj['start_idx']] + new_subj + sents[subj['end_idx']+1 : obj['start_idx']] + new_obj + sents[obj['end_idx']+1 :]
        
        dataset['sentence'][idx] = temp
        
        subject_entity.append(subj['word'])
        object_entity.append(obj['word'])
        
        subject_type.append(subj['type'])
        object_type.append(obj['type'])

    # print(len(dataset['id'], len(dataset['sentence']))
    out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],
                                'subject_entity':subject_entity,'object_entity':object_entity,
                                'subject_type':subject_type,'object_type':object_type,
                                'label':dataset['label'],})
    return out_dataset

In [103]:
def en_load_data(dataset_dir):
  # """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_ent_dataset(pd_dataset)
    return dataset

In [104]:
en_train_data_dir = "./data/en_all_train.csv"
ent_dataset = en_load_data(en_train_data_dir)

<ipython-input-102-9ebe380a9d21>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['sentence'][idx] = temp


In [105]:
ent_dataset.head()

,id,sentence,subject_entity,object_entity,subject_type,object_type,label
0,0,〈Something〉는 [E2]조지 해리슨[/E2]이 쓰고 [E1]비틀즈[/E1]가...,비틀즈,조지 해리슨,ORG,PER,no_relation
1,1,호남이 기반인 바른미래당·[E2]대안신당[/E2]·[E1]민주평화당[/E1]이 우여...,민주평화당,대안신당,ORG,ORG,no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 [E1]광주FC[/E1]는 지난 26일 [E2...,광주FC,한국프로축구연맹,ORG,ORG,org:member_of
3,3,균일가 생활용품점 (주)[E1]아성다이소[/E1](대표 [E2]박정부[/E2])는 ...,아성다이소,박정부,ORG,PER,org:top_members/employees
4,4,[E2]1967[/E2]년 프로 야구 드래프트 1순위로 [E1]요미우리 자이언츠[/...,요미우리 자이언츠,1967,ORG,DAT,no_relation


In [166]:
ent_dataset.iloc[0]['sentence']

'〈Something〉는 [E2]조지 해리슨[/E2]이 쓰고 [E1]비틀즈[/E1]가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [167]:
from transformers import AutoTokenizer
# bert-base-uncased & bert-base-cased
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
sub_obj = ["[E1]", "[/E1]","[E2]", "[/E2]"]
#dir(tokenizer)
tokenizer.add_special_tokens({
                            "additional_special_tokens": sub_obj
                            })

4

In [119]:
tokenizer.special_tokens_map # bos_token 은 뭐지?

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'additional_special_tokens': ['[E1]', '[/E1]', '[E2]', '[/E2]']}

In [ ]:
def tokenized_dataset(dataset, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    # TODO! multi sentence ; single sentence
    concat_entity = []
    for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)
        tokenized_sentences = tokenizer(
                              concat_entity,
                              list(dataset['sentence']),
                              return_tensors="pt",
                              padding=True,
                              truncation=True,
                              max_length=256,
                              add_special_tokens=True,
                              )
  return tokenized_sentences

In [168]:
out = tokenized_dataset(ent_dataset, tokenizer)

In [157]:
out

{'input_ids': tensor([[    0, 29830,     2,  ...,     1,     1,     1],
        [    0,  3772,  2139,  ...,     1,     1,     1],
        [    0,  4104, 10904,  ...,     1,     1,     1],
        ...,
        [    0, 18272,  2242,  ...,     1,     1,     1],
        [    0, 15710,     2,  ...,     1,     1,     1],
        [    0, 15437,  2242,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [172]:
out['input_ids'][0]

tensor([    0, 29830,     2,  8373, 14113,  2234,     2,   168, 30985, 14451,
         7088,  4586,   169,   793, 32002,  8373, 14113,  2234, 32003,  1504,
         1363,  2088, 32000, 29830, 32001,   543, 14879,  2440,  6711,   170,
        21406, 26713,  2076, 25145,  5749,   171,  1421,   818,  2073,  4388,
         2062,    18,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [132]:
# [E1], [/E1] 도 mask 돼야하는거 아닌가..일단 해보자..
# https://github.com/huggingface/tokenizers/issues/790
print(*out[1].tokens)
print(*out[1].special_tokens_mask)

[CLS] 민주 ##평 ##화 ##당 [SEP] 대안 ##신 ##당 [SEP] 호남 ##이 기반 ##인 바른 ##미 ##래 ##당 · [E2] 대안 ##신 ##당 [/E2] · [E1] 민주 ##평 ##화 ##당 [/E1] 이 우여곡절 끝 ##에 합당 ##해 민생 ##당 ( 가칭 ) 으로 재 ##탄 ##생 ##한다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [140]:
tokenizer("this [/E1]")

{'input_ids': [0, 25419, 32001, 2], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [134]:
from transformers import AutoConfig, AutoModel
config = AutoConfig.from_pretrained("klue/roberta-large")
backbone_model = AutoModel.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [159]:
# dir(backbone_model)

In [150]:
backbone_model.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(32000, 1024, padding_idx=1)
  (position_embeddings): Embedding(514, 1024, padding_idx=1)
  (token_type_embeddings): Embedding(1, 1024)
  (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [151]:
backbone_model.resize_token_embeddings(len(tokenizer))

Embedding(32004, 1024)

In [158]:
backbone_model.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(32004, 1024)
  (position_embeddings): Embedding(514, 1024, padding_idx=1)
  (token_type_embeddings): Embedding(1, 1024)
  (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [153]:
# 참고
# https://github.com/snoop2head/KLUE-RBERT/blob/main/models.py
# https://github.com/snoop2head/KLUE-RBERT/blob/main/train.py
# https://github.com/monologg/R-BERT/blob/master/model.py
# https://github.com/boostcampaitech2/klue-level2-nlp-13/blob/2cd72e1627a9808e87214027ed8b61cdfd10e091/utills.py
# https://github.com/boostcampaitech2/klue-level2-nlp-13/blob/2cd72e1627a9808e87214027ed8b61cdfd10e091/mytokenizers.py#L11
# https://github.com/boostcampaitech2/klue-level2-nlp-13/blob/2cd72e1627a9808e87214027ed8b61cdfd10e091/load_data.py#L76
# https://github.com/boostcampaitech2/klue-level2-nlp-13/blob/main/models.py

# https://github.com/boostcampaitech2/klue-level2-nlp-14/tree/main/solution/utils
# https://github.com/huggingface/tokenizers/issues/247